## Generate an AWS costs chart

AWS costs are downloaded via the AWS costs explorer API. The adapter creates a simpler
'costs' view which we use to build our chart.

The main logic is grouping by service but bucketing small-value services into an "other" category.

In [18]:
SELECT
  case
    WHEN service like '%Redshift%' then 'Redshift'
    WHEN service like '%Compute Cloud%' then 'EC2'
    WHEN service like '%SageMaker%' then service
    WHEN service like '%Relational Database Service%' then 'RDS'
    WHEN service like '%Simple Storage Service%' then 'S3'
    WHEN service = 'AmazonCloudWatch' then 'CloudWatch'
    WHEN service like '%CloudFront%' then 'CloudFront'
    WHEN service like '%EC2 - Other%' then 'EC2'
    ELSE 'other'
  end as svc_name,
  case 
    WHEN sum(blended) > 10000 then 10000
    ELSE sum(blended)
  end as total,  
  "end"
FROM
  awscosts.costs
WHERE
  "end" > now() - interval 30 day
GROUP BY
  svc_name, "end"
ORDER BY
  "end" desc, total desc


214 rows

svc_name,total,end
RDS,1130.09,2022-09-06
Redshift,878.43,2022-09-06
S3,584.39,2022-09-06
other,419.40,2022-09-06
CloudWatch,400.02,2022-09-06
EC2,275.21,2022-09-06
CloudFront,226.16,2022-09-06
RDS,1526.95,2022-09-05
Redshift,884.87,2022-09-05
S3,582.83,2022-09-05


In [19]:
create chart as bar_chart where x = end and stacked = true and title = 'AWS Costs'